# Training the refractive index

This notebook goes trough the main functions and objects implemented in this library. Based on a dataset containing ~4,000 entries of type (mp_id, structure, refractive index) taken from the MaterialsProject (MP). The workflow can be devided in two parts. First, the creation of a MODData object which stores the information concerning this particular dataset: the materials, the targets and optimal features. Second, a MODNetModel is trained which can later be used for predicting on unseen data.

In [1]:
import sys
from modnet.models import MODNetModel
from modnet.preprocessing import MODData

## 1. Loading the dataset

In this example the dataset is a dataframe saved as a pickle. But it can be any format as long as you can retreive the structures and targets (and the mpids optionally for fast featurization).

In [2]:
import pandas as pd
df = pd.read_pickle('data/df_ref_index.pkl')
print('{} datapoints'.format(len(df)))
df.head()

4022 datapoints


,structure,ref_index
mp-624234,"[[0.67808954 1.32800354 5.90141888] Te, [1.500...",2.440483
mp-560478,"[[-0.62755181 6.55361247 9.268476 ] Ba, [4....",1.790685
mp-556346,"[[4.43332093 4.12714801 8.8721209 ] Pr, [ 1.40...",2.056131
mp-13676,"[[-0.14481557 3.41229366 4.12618551] O, [3.2...",2.023772
mp-7610,"[[ 0.12549448 3.01287591 -0.20434955] Li, [1....",1.745509


## 2. Creating a MODData instance

### (a) structure, mpid, target creation

In [3]:
md = MODData(df['structure'],df['ref_index'].values,structure_ids = df.index, target_names = ['refractive_index'])

INFO:root:Loaded DeBreuck2020Featurizer featurizer.



If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766



### (b) Featurizing the data
The MODData has an integrated database containing the features of many materials from the MP. By enabling fast featurization they are directtly retreived from this database and not computed from the structure.

In [4]:
md.featurize(fast=True,db_file='../modnet/data/feature_database.pkl')

INFO:root:Computing features, this can take time...
INFO:root:Fast featurization on, retrieving from database...
INFO:root:Retrieved features for 4022 out of 4022 materials
INFO:root:Data has successfully been featurized!


In [5]:
md.get_featurized_df().head()

,ElementProperty|MagpieData minimum Number,ElementProperty|MagpieData maximum Number,ElementProperty|MagpieData range Number,ElementProperty|MagpieData mean Number,ElementProperty|MagpieData avg_dev Number,ElementProperty|MagpieData mode Number,ElementProperty|MagpieData minimum MendeleevNumber,ElementProperty|MagpieData maximum MendeleevNumber,ElementProperty|MagpieData range MendeleevNumber,ElementProperty|MagpieData mean MendeleevNumber,...,OPSiteFingerprint|std_dev square pyramidal CN_5,OPSiteFingerprint|std_dev trigonal bipyramidal CN_5,OPSiteFingerprint|std_dev q2 CN_11,OPSiteFingerprint|std_dev q4 CN_11,OPSiteFingerprint|std_dev q6 CN_11,OPSiteFingerprint|std_dev L-shaped CN_2,OPSiteFingerprint|std_dev water-like CN_2,OPSiteFingerprint|std_dev bent 120 degrees CN_2,OPSiteFingerprint|std_dev hexagonal pyramidal CN_7,OPSiteFingerprint|std_dev pentagonal bipyramidal CN_7
mp-624234,8.0,82.0,74.0,32.000000,30.000000,8.0,81.0,90.0,9.0,85.875000,...,0.186438,0.175091,0.021637,0.047200,0.072313,2.280295e-01,0.355493,0.217585,0.134621,0.163703
mp-560478,8.0,56.0,48.0,16.000000,10.750000,8.0,9.0,87.0,78.0,71.062500,...,0.098554,0.101200,0.029021,0.021497,0.036379,6.497400e-02,0.051046,0.253411,0.061584,0.155998
mp-556346,8.0,59.0,51.0,22.307692,19.810651,8.0,17.0,96.0,79.0,83.692308,...,0.197575,0.194990,0.048936,0.049705,0.071292,1.099133e-01,0.268237,0.282694,0.123680,0.167256
mp-13676,8.0,81.0,73.0,21.333333,19.888889,8.0,76.0,87.0,11.0,84.500000,...,0.032056,0.032056,0.046716,0.024166,0.059264,1.084202e-19,0.024395,0.199876,0.057122,0.193736
mp-7610,3.0,20.0,17.0,9.000000,4.000000,8.0,1.0,87.0,86.0,54.375000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000


### (c) Computing the optimal features

This runs the feature selction algorithm. First the multual information is computed, followed by the iterative selction based on relevance and redundancy.

This step takes time, but is normally run only once before being saved.

In [6]:
md.feature_selection(n=1100)

INFO:root:Loading cross NMI from 'Features_cross' file.
INFO:root:Starting target 1/1: refractive_index ...
INFO:root:Computing mutual information between features and target...
INFO:root:Computing optimal features...
INFO:root:Selected 50/1019 features...
INFO:root:Selected 100/1019 features...
INFO:root:Selected 150/1019 features...
INFO:root:Selected 200/1019 features...
INFO:root:Selected 250/1019 features...
INFO:root:Selected 300/1019 features...
INFO:root:Selected 350/1019 features...
INFO:root:Selected 400/1019 features...
INFO:root:Selected 450/1019 features...
INFO:root:Selected 500/1019 features...
INFO:root:Selected 550/1019 features...
INFO:root:Selected 600/1019 features...
INFO:root:Selected 650/1019 features...
INFO:root:Selected 700/1019 features...
INFO:root:Selected 750/1019 features...
INFO:root:Selected 800/1019 features...
INFO:root:Selected 850/1019 features...
INFO:root:Selected 900/1019 features...
INFO:root:Selected 950/1019 features...
INFO:root:Selected 1000

In [7]:
md.get_optimal_descriptors()[:10]

['ElementProperty|MagpieData maximum GSbandgap',
 'ElementFraction|Th',
 'CrystalNNFingerprint|std_dev hexagonal bipyramidal CN_8',
 'DensityFeatures|density',
 'ElementProperty|MagpieData avg_dev Number',
 'LocalPropertyDifference|mean local difference in Electronegativity',
 'BondOrientationParameter|mean BOOP Q l=2',
 'ElementProperty|MagpieData range NdValence',
 'DensityFeatures|packing fraction',
 'OPSiteFingerprint|mean sgl_bd CN_1']

### (d) Saving the MODData

In [8]:
md.save('out/md_ref_index')

INFO:root:Data successfully saved as out/md_ref_index!


## 3. MODNet model

In [9]:
md = MODData.load('out/md_ref_index')

INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7fe2c411a8b0> object, created with modnet version 0.1.8


### (a) Creating the MODNet

In [10]:
model = MODNetModel([[['refractive_index']]],{'refractive_index':1},n_feat=1000, num_neurons=[[128],[64],[32],[]], act='elu')
model.model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1000)]            0         
_________________________________________________________________
dense (Dense)                (None, 128)               128128    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
refractive_index (Dense)     (None, 1)                 33        
Total params: 138,497
Trainable params: 138,497
Non-trainable params: 0
_________________________________________________________________


### (b) Training the model

In [11]:
#md.shuffle()
model.fit(md,val_fraction=0.1, val_key='refractive_index', loss='mae', lr=0.001, epochs = 300, batch_size = 64, xscale='minmax',yscale=None, verbose=1)

INFO:root:Compiling model...
INFO:root:Fitting model...


Epoch 1/300
57/57 [==============================] - 0s 3ms/step - loss: 0.2584 - mae: 0.2584 - val_loss: 0.1436 - val_mae: 0.1436
Epoch 2/300
57/57 [==============================] - 0s 2ms/step - loss: 0.1385 - mae: 0.1385 - val_loss: 0.1257 - val_mae: 0.1257
Epoch 3/300
57/57 [==============================] - 0s 1ms/step - loss: 0.1241 - mae: 0.1241 - val_loss: 0.1244 - val_mae: 0.1244
Epoch 4/300
57/57 [==============================] - 0s 1ms/step - loss: 0.1155 - mae: 0.1155 - val_loss: 0.1041 - val_mae: 0.1041
Epoch 5/300
57/57 [==============================] - 0s 1ms/step - loss: 0.1082 - mae: 0.1082 - val_loss: 0.1062 - val_mae: 0.1062
Epoch 6/300
57/57 [==============================] - 0s 1ms/step - loss: 0.0960 - mae: 0.0960 - val_loss: 0.1002 - val_mae: 0.1002
Epoch 7/300
57/57 [==============================] - 0s 2ms/step - loss: 0.1055 - mae: 0.1055 - val_loss: 0.0959 - val_mae: 0.0959
Epoch 8/300
57/57 [==============================] - 0s 1ms/step - loss: 0.0941 - m

In [12]:
model.fit(md,val_fraction=0.1, val_key='refractive_index', lr=0.0005, epochs = 100, batch_size = 128, xscale='minmax',yscale=None, verbose=1)

INFO:root:Compiling model...
INFO:root:Fitting model...


Epoch 1/100
29/29 [==============================] - 0s 5ms/step - loss: 0.0018 - mae: 0.0293 - val_loss: 0.0095 - val_mae: 0.0505
Epoch 2/100
29/29 [==============================] - 0s 2ms/step - loss: 6.9871e-04 - mae: 0.0145 - val_loss: 0.0091 - val_mae: 0.0503
Epoch 3/100
29/29 [==============================] - 0s 2ms/step - loss: 5.7062e-04 - mae: 0.0126 - val_loss: 0.0092 - val_mae: 0.0503
Epoch 4/100
29/29 [==============================] - 0s 2ms/step - loss: 5.6609e-04 - mae: 0.0135 - val_loss: 0.0093 - val_mae: 0.0517
Epoch 5/100
29/29 [==============================] - 0s 2ms/step - loss: 5.0947e-04 - mae: 0.0118 - val_loss: 0.0093 - val_mae: 0.0505
Epoch 6/100
29/29 [==============================] - 0s 2ms/step - loss: 4.9527e-04 - mae: 0.0120 - val_loss: 0.0094 - val_mae: 0.0506
Epoch 7/100
29/29 [==============================] - 0s 2ms/step - loss: 4.5512e-04 - mae: 0.0109 - val_loss: 0.0098 - val_mae: 0.0532
Epoch 8/100
29/29 [==============================] - 0s 2ms

## 4. Saving the model

In [13]:
model.save('out/MODNet_refractive_index')

INFO:root:Saving model...
INFO:root:Saved model to out/MODNet_refractive_index(.json/.h5/.pkl)


## 5. Predicting on unseen data

See "predicting_ref_index" notebook